In [1]:
import numpy as np
import pandas as pd
import sklearn.preprocessing as prep
from sklearn.tree import DecisionTreeRegressor
from sklearn.linear_model import LinearRegression

In [2]:
def get_dummies_exploded(s):
    return pd.merge(s.explode().dropna(), pd.Series([1], name = 'value'), how = 'cross').pivot(columns = s.name, values = 'value').reindex(s.index).fillna(0)

In [3]:
raw = pd.read_csv('Для математики.csv', decimal=',', dtype = {
    'пол': 'category',
    'первичный очаг': 'category',
    'стадия': 'category',
    'тип кости по Mich': 'category'
})
raw.columns

Index(['ФИО', 'пол', 'возраст', 'первичный очаг', 'Метастазы', 'Лечение',
       'Химиотерапия', 'диагноз', 'срок приема ОМА (мес.)', 'тип препарата',
       'длительность заболевания (год)', 'стадия', 'тип кости по Mich'],
      dtype='object')

In [4]:
df = raw[['возраст', 'срок приема ОМА (мес.)']].join([
    pd.get_dummies(raw['пол'].apply(str.strip)),
    pd.get_dummies(raw['первичный очаг'].apply(str.strip)),
    raw['Метастазы'].apply(pd.notna),
    pd.get_dummies(raw['Лечение'].apply(lambda x: 'Лечение' if pd.notna(x) and x !='конс' else x)),
    raw['Химиотерапия'].apply(pd.notna),
    get_dummies_exploded(raw['тип препарата'].fillna('').apply(lambda x: x.replace('+', ' ').replace(',', ' ').split())),
    pd.get_dummies(raw[['стадия']]),
    pd.get_dummies(raw[['тип кости по Mich']]),
    raw['длительность заболевания (год)']
   ])